In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import gmaps
import requests
import json

# Google developer API key
from config import gkey

In [3]:
# read previous outputs into new dataframes
toronto_schools = pd.read_csv("output_data/toronto_schools.csv")
catholic_schools = pd.read_csv("output_data/toronto_catholic_schools.csv")

In [4]:
bot_ts=toronto_schools.nsmallest(10,"RATINGS")[["SCHOOL NAME","ADDRESS","FSA","RATINGS"]]
bot_cs=catholic_schools.nsmallest(10,"RATINGS")[["SCHOOL NAME","ADDRESS","FSA","RATINGS"]]

In [5]:
# Visualize bottom 10 Toronto Schools
bot_ts

,SCHOOL NAME,ADDRESS,FSA,RATINGS
197,F H Miller,300 Caledonia Road,M6E,0.7
161,First Nations School of Toronto,935 Dundas St E,M4M,1.6
96,Highland Heights,35 Glendower Circuit,M1T,1.8
124,R J Lang,227 Drewry Ave,M2M,1.9
55,Ionview,90 Ionview Road,M1K,2.0
23,Eastview,20 Waldock Street,M1E,2.3
216,Parkdale,78 Seaforth Avenue,M6K,2.8
27,West Hill,299 Morningside Avenue,M1E,3.2
31,Joseph Brant,270 Manse Road,M1E,3.2
112,Chester Le,201 Chester Le Boulevard,M1W,3.2


In [6]:
# Visualize top 10 Catholic Schools
bot_cs

,SCHOOL NAME,ADDRESS,FSA,RATINGS
39,Epiphany of our Lord,3150 Pharmacy Avenue,M1W,2.5
136,St Roch,174 Duncanwoods Drive,M9L,2.5
149,St Dorothy,155 John Garland Blvd,M9V,2.5
70,St Charles Garnier,20 Stong Court,M3N,2.8
107,Holy Family,141 Close Avenue,M6K,3.0
154,St Stephen,55 Golfdown Drive,M9W,3.3
113,Our Lady of Victory,70 Guestville Avenue,M6N,3.4
79,St Joseph,176 Leslie Street,M4M,3.6
97,Stella Maris,31 Ascot Avenue,M6E,3.7
114,Santa Maria,25 Avon Avenue,M6N,3.7


In [7]:
# read output house prices into new dataframe
house_price=pd.read_csv("output_data/toronto_api_postal_combined and filtered with price range 100K to 10M.csv")
house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
0,1,Downtown,100000,43.669567,-79.388596,#13 - 55 Bloor St,Toronto,M4W 1A6,M4W
1,2,Emery,102000,43.749187,-79.542702,#704 - 3390 Weston Rd,Toronto,M9M 2X3,M9M
2,3,Emery,104900,43.749187,-79.542702,#1104 - 3390 WESTON RD,Toronto,M9M 2X3,M9M
3,4,Mimico,105000,43.615341,-79.492996,170 THE QUEENS WAY,Toronto,M8V 2N9,M8V
4,5,Mimico,105000,43.615341,-79.492996,170 THE QUEENS WAY,Toronto,M8V 2N9,M8V


In [8]:
bot_ts_list=list(bot_ts["FSA"])
bot_cs_list=list(bot_cs["FSA"])

ts_house_price=house_price[house_price["FSA"].isin(bot_ts_list)]
cs_house_price=house_price[house_price["FSA"].isin(bot_cs_list)]

ts_house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
5,6,Parkdale,105000,43.640736,-79.434357,265 DUNN AVE,Toronto,M6K 2S1,M6K
21,22,Regal Heights,115000,43.678566,-79.439423,1075 St Clair Ave,Toronto,M6E 1A6,M6E
24,25,Parkdale,118888,43.635517,-79.441200,2901 LAKESHORE BLVD,Toronto,M6K 3C1,M6K
30,31,St. Clair West Village,119000,43.679684,-79.436317,962 St Clair Ave,Toronto,M6E 1A1,M6E
33,34,Riverside,119000,43.665714,-79.351059,623 GERRARD ST E,Toronto,M4M 1Y2,M4M


In [9]:
cs_house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
5,6,Parkdale,105000,43.640736,-79.434357,265 DUNN AVE,Toronto,M6K 2S1,M6K
9,10,Rexdale,109000,43.722031,-79.543945,# 809 - 236 ALBION RD,Toronto,M9W 6A6,M9W
10,11,Rexdale,109000,43.722031,-79.543945,#102 - 236 Albion Rd,Toronto,M9W 6A6,M9W
13,14,Rexdale,109900,43.722031,-79.543945,#912 - 236 Albion Rd,Toronto,M9W 6A6,M9W
19,20,Rexdale,115000,43.722031,-79.543945,#902 - 236 Albion Rd,Toronto,M9W 6A6,M9W


In [10]:
# find coordinates of top Toronto schools using reverse geocode by address
params = {"key": gkey,"language":"en"}
for index, row in bot_ts.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    address = row["ADDRESS"]
    params['address'] = f"{address},+Toronto,+ON"
    ts_lat_lng = requests.get(base_url, params=params)
    ts_lat_lng = ts_lat_lng.json()
    bot_ts.loc[index, "lat"] = ts_lat_lng["results"][0]["geometry"]["location"]["lat"]
    bot_ts.loc[index, "lng"] = ts_lat_lng["results"][0]["geometry"]["location"]["lng"]
  
bot_ts

,SCHOOL NAME,ADDRESS,FSA,RATINGS,lat,lng
197,F H Miller,300 Caledonia Road,M6E,0.7,43.683283,-79.458303
161,First Nations School of Toronto,935 Dundas St E,M4M,1.6,43.662086,-79.348690
96,Highland Heights,35 Glendower Circuit,M1T,1.8,43.794497,-79.300340
124,R J Lang,227 Drewry Ave,M2M,1.9,43.783793,-79.426609
55,Ionview,90 Ionview Road,M1K,2.0,43.737481,-79.274316
23,Eastview,20 Waldock Street,M1E,2.3,43.759026,-79.191444
216,Parkdale,78 Seaforth Avenue,M6K,2.8,43.642955,-79.436974
27,West Hill,299 Morningside Avenue,M1E,3.2,43.772770,-79.186684
31,Joseph Brant,270 Manse Road,M1E,3.2,43.766945,-79.175589
112,Chester Le,201 Chester Le Boulevard,M1W,3.2,43.799019,-79.331461


In [12]:
# find coordinates of top Catholic schools using reverse geocode by address
params = {"key": gkey,"language":"en"}
for index, row in bot_cs.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    address = row["ADDRESS"]
    params['address'] = f"{address},+Toronto,+ON"
    cs_lat_lng = requests.get(base_url, params=params)
    cs_lat_lng = cs_lat_lng.json()
    bot_cs.loc[index, "lat"] = cs_lat_lng["results"][0]["geometry"]["location"]["lat"]
    bot_cs.loc[index, "lng"] = cs_lat_lng["results"][0]["geometry"]["location"]["lng"]
    
bot_cs

,SCHOOL NAME,ADDRESS,FSA,RATINGS,lat,lng
39,Epiphany of our Lord,3150 Pharmacy Avenue,M1W,2.5,43.798960,-79.328864
136,St Roch,174 Duncanwoods Drive,M9L,2.5,43.753637,-79.566753
149,St Dorothy,155 John Garland Blvd,M9V,2.5,43.731679,-79.591691
70,St Charles Garnier,20 Stong Court,M3N,2.8,43.762441,-79.517749
107,Holy Family,141 Close Avenue,M6K,3.0,43.638574,-79.434615
154,St Stephen,55 Golfdown Drive,M9W,3.3,43.722093,-79.554451
113,Our Lady of Victory,70 Guestville Avenue,M6N,3.4,43.683724,-79.489810
79,St Joseph,176 Leslie Street,M4M,3.6,43.665846,-79.331681
97,Stella Maris,31 Ascot Avenue,M6E,3.7,43.679414,-79.444947
114,Santa Maria,25 Avon Avenue,M6N,3.7,43.681560,-79.477480


In [13]:
# Set coordinates for schools
coordinates_ts=bot_ts[["lat", "lng"]]
coordinates_cs=bot_cs[["lat","lng"]]

# Set coordinates for houses in Toronto school districts and Catholic school districs
coordinates_houses_ts=ts_house_price[["lat","lng"]]
coordinates_houses_cs=cs_house_price[["lat","lng"]]

In [14]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

# Customize the size of the figure
figure_layout = {'width': '1000px', 'height': '600px','border': '1px solid black','padding': '1px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

# Assign the marker layers to ts and cs variables, with T for Toronto School and C for Catholic School
marker_ts=gmaps.marker_layer(coordinates_ts,label="T",info_box_content=[f"School: {school}" for school in bot_ts["SCHOOL NAME"]])
marker_cs=gmaps.marker_layer(coordinates_cs,label="C",info_box_content=[f"School: {school}" for school in bot_cs["SCHOOL NAME"]])

# Add the layer of Toronto and Catholic school markers to the map
fig.add_layer(marker_ts)
fig.add_layer(marker_cs)

# fig

In [15]:
# Create dataset for heat layers
price_ts=ts_house_price["Price ($)"]
price_cs=cs_house_price["Price ($)"]

# Set heat layer for house price in Toronto School district with weight as house price
heat_layer_ts = gmaps.heatmap_layer(coordinates_houses_ts,weights=price_ts,dissipating=False, max_intensity=10000000,point_radius = 0.01)
fig.add_layer(heat_layer_ts)

# Set heat layer for house price in Toronto School district with weight as house price
heat_layer_cs = gmaps.heatmap_layer(coordinates_houses_cs,weights=price_cs,dissipating=False, max_intensity=10000000,point_radius = 0.01)
fig.add_layer(heat_layer_cs)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [18]:
# # We kept the option to compare School places with all the housing data

# house_locations=house_price[["lat","lng"]]
# price=house_price["Price ($)"]

# # Create a poverty Heatmap layer using all housing data
# heat_layer = gmaps.heatmap_layer(house_locations,weights=price,dissipating=False, max_intensity=10000000,point_radius = 0.01)
# fig.add_layer(heat_layer)

# fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…